In [1]:
import warnings
warnings.filterwarnings('ignore',category=FutureWarning)
warnings.filterwarnings('ignore',category=DeprecationWarning)
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time

In [2]:
df_temp = pd.read_csv("../meteo_viseu_pinhao_aveiro/dados_temperatura.csv")
df_prec = pd.read_csv("../meteo_viseu_pinhao_aveiro/dados_precipitacao.csv")
df_energy = pd.read_csv("output.csv")

In [3]:
temp_cols = df_temp.columns
prec_cols = df_prec.columns

## IPMA Temperature

In [4]:
df_temp

,Time,Temp_Viseu,Temp_Pinhao,Temp_Aveiro
0,2020-10-01 00:00:00,11.1,13.2,16.8
1,2020-10-01 00:10:00,11.1,13.2,16.7
2,2020-10-01 00:20:00,10.9,13.2,16.6
3,2020-10-01 00:30:00,11.3,13.3,16.5
4,2020-10-01 00:40:00,11.5,13.4,16.4
...,...,...,...,...
131323,2023-03-31 23:10:00,-990.0,-990.0,-990.0
131324,2023-03-31 23:20:00,-990.0,-990.0,-990.0
131325,2023-03-31 23:30:00,-990.0,-990.0,-990.0
131326,2023-03-31 23:40:00,-990.0,-990.0,-990.0


## IPMA Precipitation

In [5]:
df_prec

,Time,Precip_Viseu,Precip_Pinhao,Precip_Aveiro
0,2020-10-01 00:00:00,0.0,0.0,0.0
1,2020-10-01 00:10:00,0.0,0.0,0.0
2,2020-10-01 00:20:00,0.0,0.0,0.0
3,2020-10-01 00:30:00,0.0,0.0,0.0
4,2020-10-01 00:40:00,0.0,0.0,0.0
...,...,...,...,...
131323,2023-03-31 23:10:00,-990.0,-990.0,-990.0
131324,2023-03-31 23:20:00,-990.0,-990.0,-990.0
131325,2023-03-31 23:30:00,-990.0,-990.0,-990.0
131326,2023-03-31 23:40:00,-990.0,-990.0,-990.0


## Created Energy Dataset

In [6]:
df_energy

,Time,PT41CV,PT44AL,PT15VP,PT69QG,PT43QL,PT17RW,PT87ZW
0,2021-11-01 00:15:00,0.028,0.452,0.000,0.828,0.128,0.056,0.552
1,2021-11-01 00:30:00,0.080,0.440,0.000,0.796,0.160,0.056,0.492
2,2021-11-01 00:45:00,0.092,0.424,0.000,0.832,0.220,0.056,0.308
3,2021-11-01 01:00:00,0.040,0.408,0.000,0.800,0.404,0.088,0.156
4,2021-11-01 01:15:00,0.076,0.396,0.000,0.836,0.156,0.116,0.160
...,...,...,...,...,...,...,...,...
35027,2022-10-31 23:00:00,0.120,0.448,0.108,0.220,0.164,0.144,0.368
35028,2022-10-31 23:15:00,0.036,0.424,0.124,0.252,0.228,0.164,0.396
35029,2022-10-31 23:30:00,0.064,0.392,0.148,0.268,0.224,0.220,0.356
35030,2022-10-31 23:45:00,0.132,0.260,0.132,0.276,0.212,0.176,0.392


## Preprocessing

In [7]:
print("NaN values:\n",df_temp.isna().sum())
print("Negative values:",(df_temp.values[:,1:] < 0).sum())

NaN values:
 Time           0
Temp_Viseu     0
Temp_Pinhao    0
Temp_Aveiro    0
dtype: int64
Negative values: 41091


In [8]:
print("NaN values:\n",df_prec.isna().sum())
print("Negative values:",(df_prec.values[:,1:] < 0).sum())

NaN values:
 Time             0
Precip_Viseu     0
Precip_Pinhao    0
Precip_Aveiro    0
dtype: int64
Negative values: 16243


In [9]:
print("NaN values:\n",df_energy.isna().sum())
print("Negative values:",(df_energy.values[:,1:] < 0).sum())

NaN values:
 Time      0
PT41CV    0
PT44AL    0
PT15VP    0
PT69QG    0
PT43QL    0
PT17RW    0
PT87ZW    0
dtype: int64
Negative values: 0


### Equalize intervals

In [10]:
def resample_dataset_15_minutes(df):
    df['Time'] = pd.to_datetime(df['Time'])
    start = time.time()
    df_ind = df.set_index('Time')
    df_split = np.array_split(df_ind, len(df_ind)/6)
    df_time = df_ind.resample('15T').apply(lambda x: x[0])
    df_split15 = np.array_split(df_time, len(df_time)/4)
    for hour in range(len(df_split)):
        for col in range(df_time.shape[1]):
            df_split15[hour].iloc[0,col] = df_split[hour].iloc[0,col]
            df_split15[hour].iloc[1,col] = (df_split[hour].iloc[1,col] + df_split[hour].iloc[2,col]) / 2
            df_split15[hour].iloc[2,col] = df_split[hour].iloc[3,col]
            df_split15[hour].iloc[3,col] = (df_split[hour].iloc[4,col] + df_split[hour].iloc[5,col]) / 2
    df_concat = pd.DataFrame(np.concatenate(df_split15))
    dataframe = pd.DataFrame()
    dataframe.index = pd.date_range(start='2020-10-01', end='2023-04-01', closed='left', freq='15T')
    dataframe['Time'] = dataframe.index
    dataframe = dataframe.reset_index(drop=True)
    df_resampled = pd.concat([dataframe, df_concat], axis=1)
    print('Elapsed time: %.4f seconds' % (time.time() - start))
    return df_resampled

In [11]:
df_resampled = resample_dataset_15_minutes(df_temp)
df_resampled.columns = temp_cols
df_resampled

Elapsed time: 33.1315 seconds


,Time,Temp_Viseu,Temp_Pinhao,Temp_Aveiro
0,2020-10-01 00:00:00,11.10,13.20,16.80
1,2020-10-01 00:15:00,11.00,13.20,16.65
2,2020-10-01 00:30:00,11.30,13.30,16.50
3,2020-10-01 00:45:00,11.45,13.35,16.40
4,2020-10-01 01:00:00,10.80,13.40,16.40
...,...,...,...,...
87547,2023-03-31 22:45:00,-990.00,-990.00,-990.00
87548,2023-03-31 23:00:00,-990.00,-990.00,-990.00
87549,2023-03-31 23:15:00,-990.00,-990.00,-990.00
87550,2023-03-31 23:30:00,-990.00,-990.00,-990.00


In [12]:
df_resampled.to_pickle("df_temperature.pkl")  

In [13]:
df_resampled_prec = resample_dataset_15_minutes(df_prec)
df_resampled_prec.columns = prec_cols
df_resampled_prec

Elapsed time: 34.0281 seconds


,Time,Precip_Viseu,Precip_Pinhao,Precip_Aveiro
0,2020-10-01 00:00:00,0.0,0.0,0.0
1,2020-10-01 00:15:00,0.0,0.0,0.0
2,2020-10-01 00:30:00,0.0,0.0,0.0
3,2020-10-01 00:45:00,0.0,0.0,0.0
4,2020-10-01 01:00:00,0.0,0.0,0.0
...,...,...,...,...
87547,2023-03-31 22:45:00,-990.0,-990.0,-990.0
87548,2023-03-31 23:00:00,-990.0,-990.0,-990.0
87549,2023-03-31 23:15:00,-990.0,-990.0,-990.0
87550,2023-03-31 23:30:00,-990.0,-990.0,-990.0


## Data with more metrics

In [14]:
house1 = pd.read_csv('../meteo_viseu_pinhao_aveiro/dados_01200560.csv')
house2 = pd.read_csv('../meteo_viseu_pinhao_aveiro/dados_01210702.csv')
house1_cols = house1.columns
house2_cols = house2.columns

FileNotFoundError: [Errno 2] No such file or directory: '../meteo_viseu_pinhao_aveiro/dados_01200560.csv'

In [ ]:
house1

In [ ]:
house2

In [ ]:
print("House 1")
print("NaN values:",house1.isna().sum().sum())
print("Negative values:",(house1.values[:,2:] < 0).sum())
print("\nHouse 2")
print("NaN values:",house2.isna().sum().sum())
print("Negative values:",(house2.values[:,2:] < 0).sum())

In [ ]:
house1_resample = resample_dataset_15_minutes(house1)
house1_resample.drop(1, inplace=True)
house1_resample.shape
house1_resample.columns = house1_cols[1:]
house1_resample

In [ ]:
house2_resample = resample_dataset_15_minutes(house1)
house2_resample.drop(1, inplace=True)
house2_resample.columns = house2_cols[1:]
house2_resample